# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import pprint
# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
   
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
city_df = pd.DataFrame()
number = 1
temp =[]
humidity = []
cloudiness = []
wind_speed = []
names = []
lat = []
lng = []
country = []
for city in cities:
    try:
        base_url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}&units=Imperial"
        response = requests.get(base_url).json()
        name = response["name"]
        names.append(response["name"])
        temp.append(response["main"]["temp"])
        cloudiness.append(response["clouds"]["all"])
        humidity.append(response["main"]["humidity"])
        wind_speed.append(response["wind"]["speed"])
        lat.append(response["coord"]["lat"])
        lng.append(response["coord"]["lon"])
        country.append(response["sys"]["country"])
        print(f"Current City is #{number} and its name is {name}")
        number += 1
    except:
        print("City not found; skipping record")

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
city_df["City"] = names
city_df["Temp(F)"] = temp
city_df["Cloudiness"] = cloudiness
city_df["Humidity"] = humidity
city_df["Wind Speed (mph)"] = wind_speed
city_df["Latitude"] = lat
city_df["Longitude"] = lng
city_df["Country"] = country
city_df.to_csv("city.csv")
city_df

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
clean_city_df = city_df[city_df["Humidity"] < 100]
clean_city_df

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
clean_city_df.plot.scatter( x = "Latitude", y = "Temp(F)", title = "Latitude v Temperature Graph as of 10/08/22")
plt.savefig("Graphs/Latitude_vs_Temperature")

## Latitude vs Temperature
This graph analyzes how temperature varies with latitude. From the data points, cities at the equator (latitude = 0) are clearly higher in temperature than other cities at either end of the North/South.

## Latitude vs. Humidity Plot

In [ ]:
clean_city_df.plot.scatter( x = "Latitude", y = "Humidity", title = "Latitude v Humidity Graph as of 10/08/22")
plt.savefig("Graphs/Latitude_vs_Humidity")

## Latitude vs Humidity
This scatter plot is analyzing the humidity levels of cities with varying latitudes. Humidity seems relatively well distributed for cities not located near the equator. Cities that are near the equator tend to be pretty humid with indexes generally above 60.

## Latitude vs. Cloudiness Plot

In [ ]:
clean_city_df.plot.scatter( x = "Latitude", y = "Cloudiness", title = "Latitude v Cloudiness Graph as of 10/08/22")
plt.savefig("Graphs/Latitude_vs_Cloudiness")

## Latitude vs Cloudiness
This graph is much different than the previous graphs as cloudiness on a given day is not really a continuous value. There seems to be more cities with overcast skies with cloudiness value 100 as there are cities with clear skies at cloudiness value 0. 

## Latitude vs. Wind Speed Plot

In [ ]:
clean_city_df.plot.scatter( x = "Latitude", y = "Wind Speed (mph)", title = "Latitude v Wind Speed Graph as of 10/08/22")
plt.savefig("Graphs/Latitude_vs_Wind_Speed")

## Latitude vs Wind Speed
This graph makes sense considering most cities should not have wind speeds over 15 mph on a given day. Cities trend towards being within the 0-15 mph wind speed range with a few outliers being above that regardless of latitude. 

## Linear Regression

In [ ]:
#Separating Northern/Souther Hemisphers
north_lat_df = clean_city_df[clean_city_df["Latitude"] > 0]
south_lat_df = clean_city_df[clean_city_df["Latitude"] < 0]


####  Northern Hemisphere - Temp vs. Latitude Linear Regression

In [ ]:
north_lat_temp = linregress(north_lat_df["Latitude"],north_lat_df["Temp(F)"])
north_lat_df.plot.scatter(x ="Latitude", y = "Temp(F)", title = "Northern Hemisphere Latitude vs Temp")
plt.plot(north_lat_df["Latitude"], north_lat_temp.intercept + north_lat_temp.slope*north_lat_df["Latitude"], 'r', label='fitted line')
plt.legend()
plt.savefig("Graphs/Northern_Latitude_vs_Temperature")

## Northern Hemisphere Temperature Max
Reinforcing our idea that temperatures are hotter around the equator, this graph clearly displays that as one moves further up north cities tend to have lower and lower temperatures.

####  Southern Hemisphere - Temp vs. Latitude Linear Regression

In [ ]:
south_lat_temp = linregress(south_lat_df["Latitude"],south_lat_df["Temp(F)"])
south_lat_df.plot.scatter(x ="Latitude", y = "Temp(F)", title = "Southern Hemisphere Latitude vs Temp")
plt.plot(south_lat_df["Latitude"], south_lat_temp.intercept + south_lat_temp.slope*south_lat_df["Latitude"], 'r', label='fitted line')
plt.legend()
plt.savefig("Graphs/Southern_Latitude_vs_Temperature")


## Southern Hemisphere Temperature
Per the fitted line, as one moves closer to the equator there is a steady increase in temperature.

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
north_lat_humidity  = linregress(north_lat_df["Latitude"],north_lat_df["Humidity"])
north_lat_df.plot.scatter(x ="Latitude", y = "Humidity", title = "Northern Hemisphere Latitude vs Humidity")
plt.plot(north_lat_df["Latitude"], north_lat_humidity.intercept + north_lat_humidity.slope*north_lat_df["Latitude"], 'r', label='fitted line')
plt.legend()
plt.savefig("Graphs/Northern_Latitude_vs_Humidity")

## Northern Hemisphere Humidity
Northern Hemisphere cities on average are more humid than Southern cities based on the fitted line and amount of data points above 80 humidity index.

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
south_lat_humidity  = linregress(south_lat_df["Latitude"],south_lat_df["Humidity"])
south_lat_df.plot.scatter(x ="Latitude", y = "Humidity", title = "Southern Hemisphere Latitude vs Humidity")
plt.plot(south_lat_df["Latitude"], south_lat_humidity.intercept + south_lat_humidity.slope*south_lat_df["Latitude"], 'r', label='fitted line')
plt.legend()
plt.savefig("Graphs/Southern_Latitude_vs_Humidity")

## Southern Hemisphere Humidity
Cities closer to the equator tend to be more humid according to the fitted line.

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
north_lat_cloud  = linregress(north_lat_df["Latitude"],north_lat_df["Cloudiness"])
north_lat_df.plot.scatter(x ="Latitude", y = "Cloudiness", title = "Northern Hemisphere Latitude vs Cloudiness")
plt.plot(north_lat_df["Latitude"], north_lat_cloud.intercept + north_lat_cloud.slope*north_lat_df["Latitude"], 'r', label='fitted line')
plt.legend()
plt.savefig("Graphs/Northern_Latitude_vs_Cloudiness")

## Northern Hemisphere Cloudiness
Cities located further north tend to be more cloudy based on fitted line.

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
south_lat_cloud  = linregress(south_lat_df["Latitude"],south_lat_df["Cloudiness"])
south_lat_df.plot.scatter(x ="Latitude", y = "Cloudiness", title = "Southern Hemisphere Latitude vs Cloudiness")
plt.plot(south_lat_df["Latitude"], south_lat_cloud.intercept + south_lat_cloud.slope*south_lat_df["Latitude"], 'r', label='fitted line')
plt.legend()
plt.savefig("Graphs/Southern_Latitude_vs_Cloudiness")

## Southern Hemisphere Cloudiness
With the data set generated, as one moves closer to the equator from the south cloudiness tends to increase.

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
north_lat_wind = linregress(north_lat_df["Latitude"],north_lat_df["Wind Speed (mph)"])
north_lat_df.plot.scatter(x ="Latitude", y = "Wind Speed (mph)", title = "Northern Hemisphere Latitude vs Wind Speed")
plt.plot(north_lat_df["Latitude"], north_lat_wind.intercept + north_lat_wind.slope*north_lat_df["Latitude"], 'r', label='fitted line')
plt.legend()
plt.savefig("Graphs/Northern_Latitude_vs_Wind_Speed")


## Northern Hemisphere Wind Speed 
Data graphs northern hemisphere cities vs wind speed. Fitted line shows that most cities lie beneath 10 mph wind speeds with wind speed increasing slightly as latitude increases.

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
south_lat_wind = linregress(south_lat_df["Latitude"],south_lat_df["Wind Speed (mph)"])
south_lat_df.plot.scatter(x ="Latitude", y = "Wind Speed (mph)", title = "Southern Hemisphere Latitude vs Wind Speed")
plt.plot(south_lat_df["Latitude"], south_lat_wind.intercept + south_lat_wind.slope*south_lat_df["Latitude"], 'r', label='fitted line')
plt.legend()
plt.savefig("Graphs/Southern_Latitude_vs_Wind_Speed")

## Southern Hemisphere Latitude vs Wind Speed
Southern Hemisphere cities and the wind speeds of those cities. The fitted line states that most of the data points lie below the 10 MPH mark. There's only a slight decrease of wind speeds as latitudes get closer to the equator at the 0 latitude mark. From the data points randomly generated by our second block of code, there seems to be more Northern Hemisphere cities in the data set than Southern Hemisphere cities.